In [397]:
import numpy as np
import torch

In [398]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
display(device)

device(type='cuda')

In [399]:
np.random.seed(45)
num_samples = 40
    
# Generate data
x1 = np.random.uniform(-1, 1, num_samples)
f_x = 3*x1 + 4
eps = np.random.randn(num_samples)
y = f_x + eps

In [400]:
theta = torch.tensor([0., 0.], requires_grad=True)

In [401]:
x_tensor = torch.tensor(x1, dtype=torch.float32).reshape(-1, 1)
X = torch.cat((torch.ones(x_tensor.shape), x_tensor), 1)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

y_pred = (X @ theta).reshape(-1, 1)
loss = torch.mean((y_pred - y) ** 2)
loss.backward()

true_grad = theta.grad.clone()

display(true_grad)

tensor([-7.4471, -1.0253])

In [402]:
def compute_sgd(x_i, y_i):
    theta.grad.zero_()
    y_pred_i = x_i @ theta
    loss_i = torch.mean((y_pred_i - y_i) ** 2)
    loss_i.backward()
    
    return theta.grad.clone()

stochastic_grads = []

for i in range(y.shape[0]):
    x_i = X[i].clone().detach()
    y_i = y[i].clone().detach()
    stochastic_grads.append(compute_sgd(x_i, y_i))

mean_stochastic_grad = torch.mean(torch.stack(stochastic_grads), dim=0)

display(mean_stochastic_grad)

tensor([-7.4471, -1.0253])

In [403]:
residual_tensor = true_grad - mean_stochastic_grad
display(residual_tensor)

tensor([ 4.7684e-07, -2.3842e-07])

As we can observe from the above cell, the values in the residual tensor are all of the order $10^{-7}$, which indicates that the error is minimal. We can conclude that the stochastic gradient is a good estimate of the true gradient.